In [1]:
import cv2
import numpy as np
import os

image_folder_path = "/output_masks"

image_files = [f for f in os.listdir(image_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
image_files.sort()

print(image_files)
image_data = []

for image_file in image_files:
    image_path = os.path.join(image_folder_path, image_file)
    #print(image_path)
    image = cv2.imread(image_path)
    image_array = np.array(image[:,:,0])
    image_data.append(image_array)

image_data_array = np.array(image_data)

print("Shape of the image data array:", image_data_array.shape)

'''
print(image_data_array[0])
image = image_data_array[0]

cv2.namedWindow("Image", cv2.WINDOW_NORMAL)

# Show the image in the window
cv2.imshow("Image", image)

# Wait for a key event and close the window when a key is pressed
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

['0000.png', '0001.png', '0002.png', '0003.png', '0004.png', '0005.png', '0006.png', '0007.png', '0008.png', '0009.png', '0010.png', '0011.png', '0012.png', '0013.png', '0014.png', '0015.png', '0016.png', '0017.png', '0018.png', '0019.png', '0020.png', '0021.png', '0022.png', '0023.png', '0024.png', '0025.png', '0026.png', '0027.png', '0028.png', '0029.png', '0030.png', '0031.png', '0032.png', '0033.png', '0034.png', '0035.png', '0036.png', '0037.png', '0038.png', '0039.png', '0040.png', '0041.png', '0042.png', '0043.png', '0044.png', '0045.png', '0046.png', '0047.png', '0048.png', '0049.png', '0050.png', '0051.png', '0052.png', '0053.png', '0054.png', '0055.png', '0056.png', '0057.png', '0058.png', '0059.png', '0060.png', '0061.png', '0062.png', '0063.png', '0064.png', '0065.png', '0066.png', '0067.png', '0068.png', '0069.png', '0070.png', '0071.png', '0072.png', '0073.png', '0074.png', '0075.png', '0076.png', '0077.png', '0078.png', '0079.png', '0080.png', '0081.png', '0082.png', '00

'\nprint(image_data_array[0])\nimage = image_data_array[0]\n\ncv2.namedWindow("Image", cv2.WINDOW_NORMAL)\n\n# Show the image in the window\ncv2.imshow("Image", image)\n\n# Wait for a key event and close the window when a key is pressed\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

In [2]:
def read_all_image_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    #print(lines[4:10])
    image_data_dict = {}
    
    for i in range(4, len(lines), 2):
        current_image_id = int(lines[i].split(' ')[0])
        image_name = lines[i].split(' ')[-1]
        number_part = image_name.split(".")[0]
        index = int(number_part)
        points2d_line = lines[i + 1].split(' ')
        points2d = [(float(points2d_line[j]), float(points2d_line[j+1]), int(points2d_line[j+2])) for j in range(0, len(points2d_line), 3)]

        image_data_dict[current_image_id] = (index, points2d)

    return image_data_dict



file_path = '/sparse/images.txt'

all_image_data = read_all_image_data(file_path)

# Example
given_image_id = 393
x, points2d = all_image_data.get(given_image_id, None)

if points2d is not None:
    print(f"Points2d for image {given_image_id}: {points2d}")
else:
    print(f"Image with ID {given_image_id} not found in the file.")

print(all_image_data.keys())
print(x)

Points2d for image 393: [(949.6979370117188, 385.25555419921875, 28575), (274.657470703125, 325.3647155761719, 15003), (304.69091796875, 159.23593139648438, -1), (1018.9523315429688, 181.602294921875, 4851), (884.3744506835938, 175.2489776611328, 38346), (609.95458984375, 451.6133728027344, 14451), (1051.4188232421875, 555.794677734375, 14802), (137.55523681640625, 222.40487670898438, 6772), (713.2132568359375, 252.846435546875, 12793), (335.37139892578125, 304.5167236328125, 44531), (1122.3865966796875, 477.6331481933594, 13870), (691.4424438476562, 84.94558715820312, 6005), (438.4305419921875, 161.60955810546875, 67803), (509.04742431640625, 411.80322265625, 17732), (703.0590209960938, 612.8446655273438, 26745), (174.09564208984375, 338.33270263671875, 12797), (827.2896728515625, 276.2151794433594, 68628), (667.989013671875, 147.7235565185547, 66302), (381.7017822265625, 102.85209655761719, 19134), (581.2476196289062, 268.2110595703125, 12019), (1178.9013671875, 508.4068603515625, 24

In [3]:
from collections import Counter
from PIL import Image

input_file_path = '/sparse/points3D.txt'
output_file_path = '/results/points3D_labels.txt'

with open(input_file_path, 'r') as file:
    lines = file.readlines()

def majority_voting(labels):
    counter = Counter(labels)
    majority_label = counter.most_common(1)[0][0]
    return majority_label

def get_semantic_label(image_id, point2d_idx, point3d_id):
    idx, points2d = all_image_data.get(image_id, None)
    x, y, id_3d = points2d[point2d_idx]
    #print(id_3d, point3d_id)
    return image_data_array[idx, int(y-1), int(x-1)]

for line in lines[3:]:
    data = line.strip().split()
    point3d_id, x, y, z = int(data[0]), float(data[1]), float(data[2]), float(data[3])

    image_ids = [int(data[i]) for i in range(8, len(data), 2)]
    point2d_indices = [int(data[i + 1]) for i in range(8, len(data), 2)]

    semantic_labels = [
        get_semantic_label(image_id, point2d_idx, point3d_id)
        for image_id, point2d_idx in zip(image_ids, point2d_indices)
    ]
    majority_label = majority_voting(semantic_labels)

    output_data = [str(point3d_id), str(x), str(y), str(z), str(majority_label)]

    with open(output_file_path, 'a') as output_file:
        output_file.write(' '.join(output_data) + '\n')


In [4]:
import open3d as o3d
import numpy as np

input_file_path = '/results/points3D_labels.txt'
output_ply_path = '/results/output_visualization.ply'

class_colors = {
    "3": [128,0,0],
    "7": [255,127,80],
    "8": [255,140,0],
    "10": [255,215,0],
    "11": [184,134,11],
    "12": [189,183,107],
    "13": [128,128,0],
    "14": [255,255,0],
    "15": [154,205,50],
    "16": [0,100,0],
    "17": [0,255,0],
    "18": [144,238,144],
    "19": [46,139,87],
    "20": [32,178,170],
    "22": [47,79,79],
    "23": [70,130,180],
    "26": [30,144,255],
    "29": [25,25,112],
    "31": [0, 0, 255],
    "34": [138,43,226],
    "35": [139,0,139],
    "37": [186,85,211],
    "40": [255,192,203],
    "44": [245,245,220],
    "47": [245,222,179],
    "52": [139,69,19],
    "54": [205,133,63],
    "56": [255,228,181],
    "59": [255,240,245],
    "60": [245,255,250],
    "61": [112,128,144],
    "62": [240,255,240],
    "63": [192,192,192],
    "64": [0,0,0],
    "65": [188,143,143],
    "70": [238,130,238],
    "71": [75,0,130],
    "76": [0,128,0],
    "78": [255,165,0],
    "79": [189,183,107],
    "80": [0,128,0],
    "82": [100,149,237],
    "83": [135,206,250],
    "87": [216,191,216],
    "88": [255,228,196],
    "91": [153,50,204],
    "92": [240,255,255],
    "93": [176,224,230],
    "95": [220,220,220],
    "97": [220,20,60],
    "98": [255,69,0]
}


def get_class_color(class_label):
    return class_colors.get(str(class_label), [128, 128, 128]) 


data = np.loadtxt(input_file_path, skiprows=3, usecols=range(0, 5))  

coordinates = data[:, 1:4]
classes = data[:, -1].astype(int)  


colors = [get_class_color(class_label) for class_label in classes]

point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(coordinates)
point_cloud.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)  


o3d.io.write_point_cloud(output_ply_path, point_cloud)

print(f'PLY file saved to: {output_ply_path}')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
PLY file saved to: /home/amitabha/Downloads/CV project/output_visualization.ply


In [5]:
import numpy as np
from sklearn.neighbors import NearestNeighbors


sparse_file_path = '/results/points3D_labels.txt'
dense_file_path = '/dense/points3D.txt'
output_file_path = 'results/points3D_combined.txt'

dense_points = np.loadtxt(dense_file_path, skiprows=3, usecols=range(1, 4))

sparse_points = np.loadtxt(sparse_file_path, skiprows=3, usecols=range(1, 4))
labels = np.loadtxt(sparse_file_path, skiprows=3, usecols=4)


k_neighbors = 20

nn_model = NearestNeighbors(n_neighbors=k_neighbors, algorithm='auto').fit(sparse_points)
distances, indices = nn_model.kneighbors(dense_points)

dense_labels = np.zeros(dense_points.shape[0], dtype=int)


for i in range(dense_points.shape[0]):
    nearest_labels = labels[indices[i]]
    unique_labels, counts = np.unique(nearest_labels, return_counts=True)
    dense_labels[i] = unique_labels[np.argmax(counts)]

dense_points_with_labels = np.column_stack((dense_points, dense_labels))

header = "X Y Z Label"
np.savetxt(output_file_path, dense_points_with_labels, header=header, comments='', fmt='%1.6f')

In [6]:
import open3d as o3d
import numpy as np

input_file_path = '/results/points3D_combined.txt'
output_ply_path = '/results/output_visualization_dense.ply'

class_colors = {
    "3": [128,0,0],
    "7": [255,127,80],
    "8": [255,140,0],
    "10": [255,215,0],
    "11": [184,134,11],
    "12": [189,183,107],
    "13": [128,128,0],
    "14": [255,255,0],
    "15": [154,205,50],
    "16": [0,100,0],
    "17": [0,255,0],
    "18": [144,238,144],
    "19": [46,139,87],
    "20": [32,178,170],
    "22": [47,79,79],
    "23": [70,130,180],
    "26": [30,144,255],
    "29": [25,25,112],
    "31": [0, 0, 255],
    "34": [138,43,226],
    "35": [139,0,139],
    "37": [186,85,211],
    "40": [255,192,203],
    "44": [245,245,220],
    "47": [245,222,179],
    "52": [139,69,19],
    "54": [205,133,63],
    "56": [255,228,181],
    "59": [255,240,245],
    "60": [245,255,250],
    "61": [112,128,144],
    "62": [240,255,240],
    "63": [192,192,192],
    "64": [0,0,0],
    "65": [188,143,143],
    "70": [238,130,238],
    "71": [75,0,130],
    "76": [0,128,0],
    "78": [255,165,0],
    "79": [189,183,107],
    "80": [0,128,0],
    "82": [100,149,237],
    "83": [135,206,250],
    "87": [216,191,216],
    "88": [255,228,196],
    "91": [153,50,204],
    "92": [240,255,255],
    "93": [176,224,230],
    "95": [220,220,220],
    "97": [220,20,60],
    "98": [255,69,0]
}



def get_class_color(class_label):
    return class_colors.get(str(class_label), [128, 128, 128])  


data = np.loadtxt(input_file_path, skiprows=1, usecols=range(0, 4)) 

coordinates = data[:, 0:3]
classes = data[:, -1].astype(int)  
colors = [get_class_color(class_label) for class_label in classes]

point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(coordinates)
point_cloud.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0) 


o3d.io.write_point_cloud(output_ply_path, point_cloud)

print(f'PLY file saved to: {output_ply_path}')


PLY file saved to: /home/amitabha/Downloads/CV project/output_visualization_dense.ply
